## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

## Starting  Solution

### 1. Create spark session

In [2]:
import findspark
findspark.init('/home/anshul/spark-3.0.0-preview2-bin-hadoop2.7/')
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('customer_logistic_regression').getOrCreate()

### 2. read and see the data

In [3]:
df=spark.read.csv('customer_churn.csv',inferSchema=True,header=True)

In [4]:
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [5]:
df.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|     

In [6]:
df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

### 3. assemble the data

In [7]:
from pyspark.ml.feature import VectorAssembler

In [8]:
assembler=VectorAssembler(inputCols=['Age','Total_Purchase',
 'Account_Manager','Years',
 'Num_Sites'],outputCol='features')

In [9]:
output=assembler.transform(df)

In [11]:
output.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+--------------------+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|            features|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+--------------------+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|[42.0,11066.8,0.0...|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|[41.0,11916.22,0....|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|[38.0,12884.75,0....|
|      Phillip White|4

selecting only features and churn columns

In [12]:
final_data=output.select(['features','Churn'])

In [13]:
final_data.show()

+--------------------+-----+
|            features|Churn|
+--------------------+-----+
|[42.0,11066.8,0.0...|    1|
|[41.0,11916.22,0....|    1|
|[38.0,12884.75,0....|    1|
|[42.0,8010.76,0.0...|    1|
|[37.0,9191.58,0.0...|    1|
|[48.0,10356.02,0....|    1|
|[44.0,11331.58,1....|    1|
|[32.0,9885.12,1.0...|    1|
|[43.0,14062.6,1.0...|    1|
|[40.0,8066.94,1.0...|    1|
|[30.0,11575.37,1....|    1|
|[45.0,8771.02,1.0...|    1|
|[45.0,8988.67,1.0...|    1|
|[40.0,8283.32,1.0...|    1|
|[41.0,6569.87,1.0...|    1|
|[38.0,10494.82,1....|    1|
|[45.0,8213.41,1.0...|    1|
|[43.0,11226.88,0....|    1|
|[53.0,5515.09,0.0...|    1|
|[46.0,8046.4,1.0,...|    1|
+--------------------+-----+
only showing top 20 rows



### 4. split the data and fit the model

In [14]:
train_data,test_data=final_data.randomSplit([0.7,0.3])

In [15]:
train_data.describe().show()
test_data.describe().show()

+-------+-------------------+
|summary|              Churn|
+-------+-------------------+
|  count|                650|
|   mean|0.18153846153846154|
| stddev| 0.3857605329880039|
|    min|                  0|
|    max|                  1|
+-------+-------------------+

+-------+-------------------+
|summary|              Churn|
+-------+-------------------+
|  count|                250|
|   mean|              0.128|
| stddev|0.33475999914582455|
|    min|                  0|
|    max|                  1|
+-------+-------------------+



In [16]:
from pyspark.ml.classification import LogisticRegression

In [17]:
model=LogisticRegression(labelCol='Churn')

In [18]:
fit_model=model.fit(train_data)

In [20]:
summary=fit_model.summary

In [26]:
summary.accuracy

0.8984615384615384

In [27]:
summary.predictions.describe().show()

+-------+-------------------+------------------+
|summary|              Churn|        prediction|
+-------+-------------------+------------------+
|  count|                650|               650|
|   mean|0.18153846153846154|0.1476923076923077|
| stddev| 0.3857605329880039|0.3550679495323581|
|    min|                0.0|               0.0|
|    max|                1.0|               1.0|
+-------+-------------------+------------------+



### 5. evaluating the model

In [28]:
pred_label=fit_model.evaluate(test_data)

In [38]:
print("accuracy:",pred_label.accuracy)
print("recall:",pred_label.recallByLabel)
print("precision:",pred_label.precisionByLabel)

accuracy: 0.9
recall: [0.963302752293578, 0.46875]
precision: [0.9251101321585903, 0.6521739130434783]


current model has 90% accuracy

In [39]:
pred_label.predictions.describe().show()

+-------+-------------------+------------------+
|summary|              Churn|        prediction|
+-------+-------------------+------------------+
|  count|                250|               250|
|   mean|              0.128|             0.092|
| stddev|0.33475999914582455|0.2896057422493124|
|    min|                  0|               0.0|
|    max|                  1|               1.0|
+-------+-------------------+------------------+



### finding te area under the curve (AUC) in the ROC curve

In [42]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [43]:
eval=BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                  labelCol='Churn')

In [44]:
auc=eval.evaluate(pred_label.predictions)

In [45]:
auc

0.716026376146789

#### above is the area under the curve value

## Prediction for the new customer

In [50]:
new_model=model.fit(final_data)

In [52]:
new_customers=spark.read.csv('new_customers.csv',inferSchema=True,header=True)

In [53]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [54]:
new_final_data=assembler.transform(new_customers)

In [55]:
new_final_data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [56]:
new_result=new_model.transform(new_final_data)

In [59]:
new_result.select(['Company','prediction']).show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



#### So it predicts that we need to asign Account Manager to 
Cannon-Benson, Barron-Robertson, Sexton-Golden, Parks-Robbins